In [1]:
#IMPORTS put all here
import random
import matplotlib.pyplot as plt

from datetime import datetime

#tf imports for reading file
import numpy as np
import os
import PIL


import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow.keras import layers

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization
from tensorflow import keras


from tensorflow.keras.regularizers import l2

#extras
print(tf.__version__)
import pathlib
import shutil

2022-11-15 08:19:09.696842: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-15 08:19:09.696868: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/ayushsingh/anaconda3/envs/ai_ml_nn/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2.8.0


In [2]:
#path to image dataset
spec_path = '../content/spectrograms6sec'
chroma_path='../content/chroma6sec'
test_path='../content/testset'
genre_list=os.listdir(spec_path)


In [3]:
# os.mkdir('../content/chroma6secnew')
# dest_path='../content/chroma6secnew'

# for genre in genre_list:
#   os.mkdir(f'../content/chroma6secnew/{genre}')



# #no of samples per class
# n=3000
# cnt=0

# random.seed(123)

# for genre in genre_list:
#   genre_path=os.path.join(chroma_path,genre)
#   dest_genre_path=os.path.join(dest_path,genre)

#   #shuffling and selecting 3000 samples
#   genre_songs=os.listdir(genre_path)
#   random.shuffle(genre_songs)
#   selected_spec=genre_songs[:n]
  
#   for song in selected_spec:
#       source=os.path.join(genre_path,song)
#       dest=os.path.join(dest_genre_path,song)
#       d= shutil.copyfile(source, dest)


# EQUALISING DATASET

In [4]:
#loading data_sets
spec_path='../content/spectrograms6secnew/'
chroma_path='../content/chroma6secnew/'

data_dir_spec=pathlib.Path(spec_path)
data_dir_chroma=pathlib.Path(chroma_path)


In [5]:
#adding dataset
batch_size=64
image_height=100
image_width=200


In [6]:
#training dataset spectrograms
train_ds_spec = tf.keras.utils.image_dataset_from_directory(
  data_dir_spec,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(image_height, image_width),
  batch_size=batch_size)

# training dataset chromagrams

train_ds_chroma = tf.keras.utils.image_dataset_from_directory(
  data_dir_chroma,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(image_height, image_width),
  batch_size=batch_size)

Found 30000 files belonging to 10 classes.
Using 24000 files for training.


2022-11-15 08:19:16.305613: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-15 08:19:16.305638: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-15 08:19:16.305655: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (CoderCatA5Pop): /proc/driver/nvidia/version does not exist
2022-11-15 08:19:16.305884: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Found 30000 files belonging to 10 classes.
Using 24000 files for training.


In [7]:
val_ds_spec = tf.keras.utils.image_dataset_from_directory(
  data_dir_spec,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(image_height, image_width),
  batch_size=batch_size)

val_ds_chroma = tf.keras.utils.image_dataset_from_directory(
  data_dir_chroma,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(image_height, image_width),
  batch_size=batch_size)


Found 30000 files belonging to 10 classes.
Using 6000 files for validation.
Found 30000 files belonging to 10 classes.
Using 6000 files for validation.


In [8]:
for image_batch, labels_batch in train_ds_spec:
  print(image_batch.shape)
  print(labels_batch.shape)
  break
for image_batch, labels_batch in train_ds_chroma:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(64, 100, 200, 3)
(64,)
(64, 100, 200, 3)
(64,)


In [9]:
class_names = train_ds_spec.class_names


In [10]:
AUTOTUNE = tf.data.AUTOTUNE

#
train_ds_spec = train_ds_spec.cache().prefetch(buffer_size=AUTOTUNE)
val_ds_spec = val_ds_spec.cache().prefetch(buffer_size=AUTOTUNE)


#
train_ds_chroma = train_ds_chroma.cache().prefetch(buffer_size=AUTOTUNE)
val_ds_chroma = val_ds_chroma.cache().prefetch(buffer_size=AUTOTUNE)



In [11]:
%load_ext tensorboard

# TRAINING SPECTOGRAMS

In [11]:
#layer config 1 for spec

num_classes = len(class_names)
model = Sequential([
  layers.Rescaling(1./255, input_shape=(image_height, image_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes,kernel_regularizer=l2(0.1),activation = "linear")
])

#hyperparams
opt = keras.optimizers.Adam(learning_rate=0.0003)

model.compile(optimizer=opt,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()

epochs=20
filename=datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = "logs/scalars/" + filename
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

model.fit(
    train_ds_spec,
    validation_data=val_ds_spec,
    epochs=epochs,
    verbose=1,
    callbacks=[tensorboard_callback],
    )

model.save(filename)



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 100, 200, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 100, 200, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 50, 100, 16)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 50, 100, 16)       2320      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 25, 50, 16)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 25, 50, 32)        4

2022-11-15 07:34:59.600701: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: 20221115-070459/assets


In [12]:
#layer config 2 for spec

num_classes = len(class_names)
model = Sequential([
  layers.Rescaling(1./255, input_shape=(image_height, image_width, 3)),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(8, 4, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes,kernel_regularizer=l2(0.1),activation = "linear")
])
#hyperparams
opt = keras.optimizers.Adam(learning_rate=0.0003)
model.compile(optimizer=opt,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()

epochs=15
filename=datetime.now().strftime("%Y%m%d-%H%M%S")

logdir = "logs/scalars/" + filename
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

model.fit(
    train_ds_spec,
    validation_data=val_ds_spec,
    epochs=epochs,
    verbose=1,
    callbacks=[tensorboard_callback],
    )

model.save(filename)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 100, 200, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 100, 200, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 100, 200, 16)      4624      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 50, 100, 16)      0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 50, 100, 8)        2056      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 25, 50, 8)        0         
 2D)                                                    

2022-11-15 09:52:02.374911: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: 20221115-081927/assets


In [13]:
#layer config 3 for spec
num_classes = len(class_names)
model = Sequential([
  layers.Rescaling(1./255, input_shape=(image_height, image_width, 3)),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes,kernel_regularizer=l2(0.1),activation = "linear")
])
#hyperparams
opt = keras.optimizers.Adam(learning_rate=0.0003)

filename=datetime.now().strftime("%Y%m%d-%H%M%S")
model.compile(optimizer=opt,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()

epochs=20

logdir = "logs/scalars/" + filename
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

model.fit(
    train_ds_spec,
    validation_data=val_ds_spec,
    epochs=epochs,
    verbose=1,
    callbacks=[tensorboard_callback],
    )

model.save(filename)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_1 (Rescaling)     (None, 100, 200, 3)       0         
                                                                 
 conv2d_3 (Conv2D)           (None, 100, 200, 32)      896       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 50, 100, 32)      0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 50, 100, 32)       9248      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 25, 50, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 25, 50, 32)       

# TRAINING CHROMATOGRAMS


In [11]:
#layer config 1 for chroma

num_classes = len(class_names)
model = Sequential([
  layers.Rescaling(1./255, input_shape=(image_height, image_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes,kernel_regularizer=l2(0.1),activation = "linear")
])

#hyperparams
opt = keras.optimizers.Adam(learning_rate=0.0003)

model.compile(optimizer=opt,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()

epochs=20
filename=datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = "logs/scalars/" + filename
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

model.fit(
    train_ds_chroma,
    validation_data=val_ds_chroma,
    epochs=epochs,
    verbose=1,
    callbacks=[tensorboard_callback],
    )

model.save(filename)



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 100, 200, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 100, 200, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 50, 100, 16)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 50, 100, 16)       2320      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 25, 50, 16)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 25, 50, 32)        4

2022-11-15 08:06:57.755999: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: 20221115-073750/assets


In [ ]:
#layer config 2 for chroma

num_classes = len(class_names)
model = Sequential([
  layers.Rescaling(1./255, input_shape=(image_height, image_width, 3)),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(8, 4, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes,kernel_regularizer=l2(0.1),activation = "linear")
])
#hyperparams
opt = keras.optimizers.Adam(learning_rate=0.0003)
model.compile(optimizer=opt,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()

epochs=20
filename=datetime.now().strftime("%Y%m%d-%H%M%S")

logdir = "logs/scalars/" + filename
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

model.fit(
    train_ds_chroma,
    validation_data=val_ds_chroma,
    epochs=epochs,
    verbose=1,
    callbacks=[tensorboard_callback],
    )
model.save(filename)

In [ ]:
#layer config 3 for chroma
num_classes = len(class_names)
model = Sequential([
  layers.Rescaling(1./255, input_shape=(image_height, image_width, 3)),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),s
  layers.MaxPooling2D(),
  
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes,kernel_regularizer=l2(0.1),activation = "linear")
])
#hyperparams
opt = keras.optimizers.Adam(learning_rate=0.0003)

filename=datetime.now().strftime("%Y%m%d-%H%M%S")
model.compile(optimizer=opt,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()

epochs=20

logdir = "logs/scalars/" + filename
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

model.fit(
    train_ds_chroma,
    validation_data=val_ds_chroma,
    epochs=epochs,
    verbose=1,
    callbacks=[tensorboard_callback],
    )
model.save(filename)